## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Открытие файла с данными и изучение общей информации

In [1]:
# импорт библиотек
import pandas as pd
from pymystem3 import Mystem
from nltk.stem import SnowballStemmer 

Чтение файла даных

In [2]:
# датасет
try:
    data = pd.read_csv('d:\Home\datasets\data.csv')
except:
    data = pd.read_csv('/datasets/data.csv')

Изучение общей инофрмации, структура таблицы

In [3]:
# получение общей информации о структуре и типах данных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Получение имен столбцов

In [4]:
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Содержимое таблицы (визуальное знакомство)

In [5]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Количество уникальных идетификаторов в толбцах

In [6]:
data.nunique()

children                8
days_employed       19351
dob_years              58
education              15
education_id            5
family_status           5
family_status_id        5
gender                  3
income_type             8
debt                    2
total_income        19351
purpose                38
dtype: int64

Число уникальных значений по значащим признакам: число детей, среднее , медиана

In [7]:
print(data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


In [8]:
print('среднее количество детей',data['children'].mean())

среднее количество детей 0.5389082462253194


In [9]:
print('медиана количества детей',data['children'].median())

медиана количества детей 0.0


Число уникальных значений по значащим признакам: образование

In [10]:
print(data['education'].unique())

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']


Число уникальных значений по значащим признакам: семейное положение

In [11]:
print(data['family_status'].value_counts())

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


Число уникальных значений по значащим признакам: пол

In [12]:
print(data['gender'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


Число уникальных значений по значащим признакам: тип занятости

In [13]:
print(data['income_type'].unique())

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


Число уникальных значений по значащим признакам: цель кредита

In [14]:
print(data['purpose'].value_counts())

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

Суммарное количество ненулевых элементов по столбцам

In [15]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Изучим содержимое строки, где не указан гендерный признак. Возможно восстановить руками

In [16]:
data.loc[data['gender'] == 'XNA'] 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Выведем суммарное количество уникальные имена family_status. Общее число элементов

In [17]:
print(data['family_status'].value_counts())

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


In [18]:
print(data['family_status'].value_counts().sum())

21525


Посмотрим столбец days_employed, есть ли пропуски, число рабочих дней должно быть положительным или раным нулю , если стаж в годах равен нулю. В столбце были замечены отрицательные числа и пропуски

In [19]:
print(data['days_employed'].isnull().sum())

2174


In [20]:
data['days_employed'].ge(0).sum()

3445

### Вывод

1. Данныев столбцах представлены строковыми, целочисленные, вещественые типами.
2. Набор данных из 21 525 строк, 12 столбцов
3. Названия столбцов в нижем регистре, без пробелов - удовлетворяют условиям для анализа содержимого таблицы
4. Столбец 'education' имеет дубликаты - верхний/нижний регистры. Необходима замена на написание в одном регистре. Скорее  всего этого хватит для дальнейшей обработки
5. В столбцах 'total_income'  имеются пропуски. Удаление приведет к искажению выводов, пропуски 10% выборки. Решение - заполнение средним по групе по признаку тип занатости 'incom_type'
6. Имеются дубли в столце 'purpose'. Например 'покупка жилья' и 'покупка жилья для семьи'. Статья 'операции с жильем' возможно  это аренда, заменять нет необходимости - на результат задачи не влияет
7. Столбец 'gender' имеет 3 уникальных имени, необходимо удалить значение XNA. Значение носит случайный характе, обусловленный человеческим фактором. Количество со  значение XNA  - 1 ,  возможно удалить. Восстановить признак по другим признакам (семейное положение) не удалось
8. Столбец 'days_employed' содержит вещественные данные со знаком, кроме этого число дней - целый тип данных. Необходима бработка пропусков, замена знака, замена данных со значениями более 0 на среднее. Заполнение будет по среднему в группе. Группировка производится по возрастному признаку. Положительные числа в исходной таблицу это чило рабочих часов за период рабочего стажа, а не ошибка. Величны примерно соответствует количеству рабочих часов , то есть дни = значение /24  Погрешность объясняется возможно графиком по 12 часов, вахтовым методом, и не учитыванием праздничных дней
9. Столбец 'family_stat us' необходимо привести к нижнему регистру. Для дальнейшего анализа признак 'разведен' можно объединить с признаком 'не женат', а признак 'гражданский брак' объединить с признаком 'женат'. Почему не включен признак 'вдовец' - отсутствие собственого желания в текущем положении.  
10. Признак чило детей нуждается к корректировке, отрицательных значений быть не должно,  количество 20 детей вероятно опечатка, либо 2, либо 0 (человеческий фактор), заменим медианным значением учитывая число вхождений заемщиков, у которых указано 20 детей

### Шаг 2. Предобработка данных

### Обработка пропусков


<span style="color:blue"> Замена значений ежемесячного дохода в столбце  total_income == Nan на значения ежемесячного дохода равного среднегрупповому значению. Группируем по типу занятости.  Проверка результата</span> 

In [21]:
for row in data['income_type'].unique():
    group_income = data.loc[data.loc[:, 'income_type'] == row]
    total_income_median = group_income['total_income'].median()
    data.loc[data['income_type'] == row, 'total_income'] = group_income['total_income'].fillna(total_income_median)
# Этот вариант красивее 
# data['total_income'] = data.groupby('income_type')['total_income'].apply(lambda x: x.fillna(x.median()))    

In [22]:
data['total_income'].isnull().sum()

0

### Заполнение в признаке число рабочих дней

1. Заменим всплески (это ничто иное как число рабочих часов, а не дней). Значения разделим на 24

In [23]:
group_days = data.loc[data.loc[:, 'days_employed'] > 0]
data.loc[data['days_employed'] > 0, 'days_employed'] =  group_days['days_employed'] / 24

2. Преобразуем отрицательные. Возьмем abs

In [24]:
data['days_employed'] = data['days_employed'].abs()

3. Заменим пустые значения признака количества рабочих дней средним по группе тип занятости. Если сначала заменить, некорректное значение получим ввиду наличия в признаке больших чиел и отрицательных

In [25]:
for row in data['income_type'].unique():
    group_days_income = data.loc[data.loc[:, 'income_type'] == row]
    days_mean = group_days_income['days_employed'].mean()
    data.loc[data['income_type'] == row, 'days_employed'] = group_days_income['days_employed'].fillna(days_mean)

Проверка отсутствия пустых строк

In [26]:
data['days_employed'].isnull().sum()

0

Замена пропусков  в столбце 'gender'. 

In [27]:
data.loc[(data['gender'] == 'XNA'), 'gender'] = 'undef'
data['gender'].unique()

array(['F', 'M', 'undef'], dtype=object)

### Замена в признаке чило детей

Количество детей = -1, ошибка, заменим на 1

In [28]:
data['children'] = data['children'].abs()

Возможно количество детей 20 ошибка заполнения данных. С другой стороны возможно, что внебрачные дети, условиями не оговорено что указаны только совместные дети

In [29]:
print(data['children'].value_counts())  

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64


Порверка всей таблицы на наличие пропусков

In [30]:
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

  
1. Заменили пустые значения в столбце days_employed Nan на значения средне группового по признаку типу занятости. Количества рабочих дней, равное 0, не влияет на цель исследований
    
2. Заменили пустые значения признака дохода в  столбце total_income Nan на значение среднегруппового по типу занятости. Сумма дохода скорее влияет на срок выплаты. Удалять или заполнять 0 нельзя
    
3. Заменили в признаке genger пропуски на 'undef'. По косвенным признакам отнести к какому-либо полу не удалось
    
4. В признаке число детей значение -1 заменили на 1, Возможно опечатка или ошибка выгрузки.
    
5. В признаке число детей значение 20 замены не поддается, удалим при разборе дубликатов. Из всей выборки процент небольшой, а значения числа детей 20 физически невозможно
    
6. Все пропуски заполнены можно преходить к обработке типов для дальнейшего анализа 


### Замена типа данных

In [31]:
data['days_employed'] = data['days_employed'].astype('int')
data['children'] = data['children'].astype('int')

### Вывод

1. Поскольку по смыслу количество дней это целочисленный параметр, произведена замена типа на int
2. Для дальнейшего анализа удобно использовать тип str в значениях столбцов family_status, education, income_type, family_status, purpose.

### Обработка дубликатов

Перевод в нижний регистр значений в признаке образование 'education' и семейное положение 'family_status'

In [32]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()

Проверка дубликатов по фрэйму данных. Удаление с переиндексированием. Проверка

In [33]:
print(data.duplicated().sum())

71


In [34]:
data = data.drop_duplicates().reset_index(drop=True)

In [35]:
print(data.duplicated().sum())

0


### Вывод

1. После преобразования значений признака образование в нижний регистр  и преобразроавнии в значениях признака количество детей, дубликатов в даных нашлой в количестве 71 
2. Дубликкаты успешно удалены из таблицы

### Лемматизация

Если в строка после лемматизации на входе сожержит какой либо из указанных признаков, то она его возвращает

In [36]:
def parser_purpose(row):
    if 'свадьба' in row:         
        return('свадьба')
    if ('недвижимость' in row) or ('жилье' in row):        
        return('недвижимость')
    if 'автомобиль' in row:
        return('автомобиль')
    if 'образование' in row:
        return ('образование')


In [37]:
m = Mystem()
lemmas = []

Создаем новый признак "purpose_pars" для цели кредита, куда будет заносить результат разбора лемматизации. Например цели 'автомобиль', 'купить автомобиль', 'ремонт автомобиля' одна категория. Разбор осуществим в цикле по первоначальному признаку цель 

In [ ]:
data['purpose_pars'] = data['purpose'] 
data["purpose_pars"] = data["purpose"].apply(m.lemmatize)

In [ ]:
data["purpose_pars"] = data["purpose_pars"].apply(parser_purpose)
data.head(2)

### Вывод

В признаке цель кредита обнаружилось множественное совпадение данных. Ручное исправлоение долгий процесс. Использовали библиотеку для разбиения на леммы текста в признаки 'purpose' Перечень лемм счетный, здесь допускаем ручную обработку c помощью функции



### Категоризация данных

#### Зависимость просрочек от уровня дохода. Разобъем на 3 группы:  низкий уровень дохода, средний, высокий 


Процент должников

In [ ]:
data['debt'].mean()

выделение категорий в зависимости от доходов, используем квантили

In [ ]:
def total_income_gr(total_income):
    if total_income <= data['total_income'].quantile(0.25):
        return 'низкий доход'
    if total_income <= data['total_income'].quantile(0.5):
        return 'средний доход'
    if total_income <= data['total_income'].quantile(0.75):
        return 'высокий доход'       
    return 'очень высокий доход'

In [ ]:
data['total_incomt_gr'] = data['total_income'].apply(total_income_gr)
data.groupby('total_incomt_gr')['debt'].agg(['mean','count']).style.format({'mean': '{:.2%}'})

Зависимость просрочек от возраста. Разобъем на 3 группы по стажу работы - молодые, средний возраст, старший возраст

In [ ]:
data['dob_years_gr'] = pd.cut(data['dob_years'],3)
data.groupby('dob_years_gr')['debt'].agg(['mean','count']).style.format({'mean': '{:.2%}'})

### Вывод

Все группы (низкий доход,  средний доход, высокий доход) имели просрочку в выплате кредита около 8%.
Клиенты с низким  и средним уровнем дохода имеют примерно одинаковый процент просрочки, около среднего.
Процентдолжников клиентов с высоким доходом  выше среднего процента, однако абслоютное значени должников этой группы значительно меньше  в двух других группах.
Клиенты с низким дохом составляют  большинство должников в абсолютном значении.
Среди разных возрастных групп число должников падает с увеличением возраста заемщика.
Процент должников среди первой возрастной группы боллше среднего, а в великовозрастной группе процент ниже среднего

### Шаг 3. Ответьте на вопросы

Посмотрим как зависит возврат кредита в срок от условия наличие детей 

In [ ]:
data.pivot_table(index='children',values='debt',aggfunc=['mean','count'])

### Вывод

Доля заемщиков без детей от общего числа неплательщиков самая высокая.
С увеличением числа детей,  число заемщиков уменьшается.
Доля неплательщиков внутри групп, примерно одинаковая

Рассмотрим как семейное положение влияет на наличе просрочек

In [ ]:
data.groupby('family_status')['debt'].agg(['mean','count'])

### Вывод

Процент по группам примерно равен среднему проценту должников от общего количества должников
В абсолютном значении просрочке подвержены женатые в браке или гражданском браке заемщики
Люди в разводе или вдовцы наименее склонны к задолженности. Маленький процент среди группы и в абсолютном значении число невелико

Рассмотрим как образование заемщика влияет на наличе просрочек

In [ ]:
data.groupby('education')['debt'].agg(['mean','count', 'sum']).style.format({'mean': '{:.2%}'})

### Вывод

1. Явно видна связь между уровнем образования и наличием долга. Чем выше уровень образщоавния, тем процент должников меньше в каждой группе.
2. В абсолютном значении наиболее подвержены к долгу люди со среднем  и высшим образованием. Среди групп с неполным высшим или начальным заемщиков мало.

Рассмотрим как разные цели кредита влияют на его возврат в срок

In [ ]:
data.groupby('purpose_pars')['debt'].agg(['mean','count']).style.format({'mean': '{:.2%}'})

### Вывод

Процент в каждой группе близок к среднему процену должников. Доля неплательщтков по кредитам, связанными с недвижимиостью  от общего числа заемщиков, самая высокая. Видимо потому что сумма кредита самая высокая. Однако процент среди группы самый низкий. Наименее подвержены просрочкам заемы на цели свадьбы

### Шаг 4. Общий вывод

1. Замена пропусков в данных признака дохода заемщика в столбце  total_income была произведена  на значения ежемесячного дохода равного среднегрупповому значению. Значения в группы выделены по типу занятости. Пропуски связаны скорее с выгрузкой данных, потому что величина дохода ключевой показатель для выдачи кредита
2. Замены очень больших значений признака числа рабочих дней произведена на значения разделенные на 24 (это ничто иное как число рабочих часов, а не дней). Погрешность в рабочих днях небольшая. Данные вероятно внеслись некоректно человеком, производщим заполнение. Здесь же, число дней взято по модулю, для получения положительных значений , веротяно неправильная выгрузка
3. ПроизведенаСамый неблагоприятный  тип заемщика - человек среднего возраста, со средним образованием, с достатком ниже среднего, женатый или в гражданском браке бездетный, берущий кредит на любые цели , связанные с недвижимостью 
4. Произведена замена в признаке genger пропусков на 'undef'. По косвенным признакам отнести к какому-либо полу не удалось
5. В признаке число детей значение -1 заменили на 1, Возможно опечатка или ошибка выгрузки.  
6. Доля заемщиков без детей от общего числа неплательщиков самая высокая и с увеличением числа детей,  число заемщиков уменьшается. Доля неплательщиков внутри групп, примерно одинаковая
7. Просрочкам в уплате кредита в абсолютном значении  подвержены женатые в браке или гражданском браке заемщики, однако процент по группам примерно одинаковый. Наименее склонны к просрочкам люди в разводе или вдовцы 
8. Явно видна связь между уровнем образования и наличием долга. Чем выше уровень образоавния, тем процент должников меньше в каждой группе.  В абсолютном значении наиболее подвержены к долгу люди со среднем  и высшим образованием. Заемщикик с неполным высшим или начальным образованием не склонны к кредитам.
9.  Наименее подвержены просрочкам заемы на цели свадьбы, доля неплательщтков по кредитам, связанными с недвижимиостью  от общего числа заемщиков, самая высокая. Видимо потому что сумма кредита самая высокая. Однако процент среди группы самый низкийю Процент в каждой группе близок к среднему процену должников
10. Самый неблагоприятный  тип заемщика - человек среднего возраста, со средним образованием, с достатком ниже среднего, женатый или в гражданском браке бездетный, берущий кредит на любые цели , связанные с недвижимостью 